<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
# <font color='blue'>Data Science Academy</font>
## <font color='blue'>Deep Learning Para Aplicações de Inteligência Artificial com Python e C++</font>
## <font color='blue'>Projeto 4</font>
## <font color='blue'>Fine Tuning de LLM Open-Source com Dataset Customizado Para Geração de Texto</font>

## Instalando e Carregando os Pacotes

In [1]:
!pip install -q -U watermark

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 77.3 MB/s eta 0:00:00


In [2]:
!pip install -q accelerate peft bitsandbytes transformers trl datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.3/366.3 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
# Imports
import accelerate
import peft
import bitsandbytes
import transformers
import trl
import datasets

In [5]:
# Imports
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from transformers import TrainingArguments
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTConfig, SFTTrainer
import warnings
warnings.filterwarnings('ignore')

In [6]:
%reload_ext watermark
%watermark -a "Data Science Academy"

Author: Data Science Academy



In [7]:
# Verifica o modelo da GPU
if torch.cuda.is_available():
    print('Número de GPUs:', torch.cuda.device_count())
    print('Modelo GPU:', torch.cuda.get_device_name(0))
    print('Total Memória [GB] da GPU:',torch.cuda.get_device_properties(0).total_memory / 1e9)

Número de GPUs: 1
Modelo GPU: NVIDIA A100-SXM4-40GB
Total Memória [GB] da GPU: 42.474471424


In [8]:
# Reset da memória da GPU (quando necessário)
from numba import cuda
device = cuda.get_current_device()
device.reset()

## Carregando o Dataset

https://huggingface.co/datasets/nlpie/Llama2-MedTuned-Instructions

In [9]:
# Carrega o dataset
dsa_dataset = load_dataset("nlpie/Llama2-MedTuned-Instructions")

README.md:   0%|          | 0.00/2.96k [00:00<?, ?B/s]

(…)-00000-of-00001-a8790d88efc2bc45.parquet:   0%|          | 0.00/91.1M [00:00<?, ?B/s]

(…)-00000-of-00001-b543c64b1786c03e.parquet:   0%|          | 0.00/6.08M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/200252 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/70066 [00:00<?, ? examples/s]

In [10]:
# Visualiza o dataset
dsa_dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'source'],
        num_rows: 200252
    })
    validation: Dataset({
        features: ['instruction', 'input', 'output', 'source'],
        num_rows: 70066
    })
})

In [11]:
# Loop para visualizar os 3 primeiros registros
for i in range(3):
    data = dsa_dataset['train'][i]
    print(f"Data Point {i + 1}:")
    print("Instruction:", data['instruction'])
    print("Input:", data['input'])
    print("Output:", data['output'])
    print("\n-----------------------------\n")

Data Point 1:
Instruction: In your role as a medical professional, address the user's medical questions and concerns.
Input: My relative suffering from secondary lever cancer ( 4th stage as per Allopathic doctor) and primary is in rectum. He is continuously with 103 to 104 degree F fever. Allpathic doctor suggested chemo only after fever subsidises. Is treatment possible at Lavanya & what is the time scale of recover.
Output: Hi, dairy have gone through your question. I can understand your concern. He has rectal cancer with liver metastasis. It is stage 4 cancer. Surgery is not possible at this stage. Only treatment options are chemotherapy and radiotherapy according to type of cancer. Inspite of all treatment prognosis is poor. Life expectancy is not good. Consult your doctor and plan accordingly. Hope I have answered your question, if you have any doubts then contact me at bit.ly/ Chat Doctor. Thanks for using Chat Doctor. Wish you a very good health.

-----------------------------



## Trabalhando com Amostras Menores (Para Acelerar o Tempo de Treino do Modelo)

In [12]:
# Amostra de treino
dsa_dataset["train"] = dsa_dataset["train"].select(range(3500))

In [13]:
# Amostra de teste
dsa_dataset["test"] = dsa_dataset["train"].select(range(300))

In [14]:
# Visualiza o dataset
dsa_dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'source'],
        num_rows: 3500
    })
    validation: Dataset({
        features: ['instruction', 'input', 'output', 'source'],
        num_rows: 70066
    })
    test: Dataset({
        features: ['instruction', 'input', 'output', 'source'],
        num_rows: 300
    })
})

## Ajustando o Formato do Prompt de Entrada Para o LLM

In [15]:
# Função para criação do prompt
def dsa_cria_prompt(sample):
    prompt = sample['instruction']
    prompt += sample['input']
    single_turn_prompt = f"Instruction: {prompt}<|end_of_turn|>AI Assistant: {sample['output']}"
    return single_turn_prompt

In [16]:
# Exemplo de uso da função
dsa_cria_prompt(dsa_dataset["train"][0])

"Instruction: In your role as a medical professional, address the user's medical questions and concerns.My relative suffering from secondary lever cancer ( 4th stage as per Allopathic doctor) and primary is in rectum. He is continuously with 103 to 104 degree F fever. Allpathic doctor suggested chemo only after fever subsidises. Is treatment possible at Lavanya & what is the time scale of recover.<|end_of_turn|>AI Assistant: Hi, dairy have gone through your question. I can understand your concern. He has rectal cancer with liver metastasis. It is stage 4 cancer. Surgery is not possible at this stage. Only treatment options are chemotherapy and radiotherapy according to type of cancer. Inspite of all treatment prognosis is poor. Life expectancy is not good. Consult your doctor and plan accordingly. Hope I have answered your question, if you have any doubts then contact me at bit.ly/ Chat Doctor. Thanks for using Chat Doctor. Wish you a very good health."

In [17]:
# Exemplo de uso da função
dsa_cria_prompt(dsa_dataset["train"][6])

"Instruction: Given your profession as a doctor, please provide responses to the medical questions using the patient's description.\nAnalyze the question given its context. Give both long answer and yes/no decision.###Question: Are serum pepsinogens and Helicobacter pylori associated with esophageal squamous cell carcinoma in a high-risk area in China?\n###Context: The role of serum pepsinogen level and Helicobacter pylori infection in esophageal carcinoma remains controversial. It may be a risk or protective factor, or without association with esophageal carcinoma. We prospectively examined associations between serum pepsinogen status, H pylori infection and the risk of esophageal squamous cell carcinoma in the Chinese population. In the present study, 1501 subjects from a community-based general population of Northern China were included. The incidence of esophageal carcinoma among the subjects was registered during a 15-year follow-up period by annual home visit, and the risks of lo

## Parâmetros de Quantização

Quantização de LLM (Large Language Model) é um processo de reduzir a precisão dos parâmetros de um modelo de linguagem grande (LLM) para um valor mais baixo, sem perder muito da sua performance. Isso é feito para reduzir a complexidade do modelo e, consequentemente, melhorar a eficiência computacional e a eficiência de armazenamento.

A quantização de LLM é útil em várias situações, como:

- Redução do tamanho do modelo: a quantização pode reduzir o tamanho do modelo, o que pode melhorar a eficiência de armazenamento e transferência de dados.

- Melhoria da eficiência computacional: a quantização pode melhorar a eficiência computacional do modelo, tornando-o mais rápido e mais eficiente em termos de recursos computacionais.

- Aumento da escalabilidade: a quantização pode permitir que os modelos sejam treinados em hardware mais barato e mais eficiente, o que pode ser útil em aplicações de produção.

BitsAndBytesConfig é usado para configurar a quantização de LLM. Ele é um objeto que contém informações sobre a quantização, como:

- Tipo de quantização: o tipo de quantização a ser usado, como quantização de ponto flutuante ou quantização de inteiro.

- Número de bits: o número de bits a ser usado para representar os parâmetros do modelo.

O BitsAndBytesConfig é usado para configurar a quantização de LLM e pode ser usado em conjunto com outros parâmetros para treinar um modelo de linguagem mais eficiente e escalável.

In [18]:
# Define os parâmetros de quantização
dsa_bnb_config = BitsAndBytesConfig(load_in_4bit = True,
                                    bnb_4bit_quant_type = "nf4",
                                    bnb_4bit_compute_dtype = "float16",
                                    bnb_4bit_use_double_quant = True)

## Carregando o LLM e o Tokenizador

https://huggingface.co/berkeley-nest/Starling-LM-7B-alpha

In [19]:
# Nome do repositório no Hugging Face
repositorio_hf = "berkeley-nest/Starling-LM-7B-alpha"

In [20]:
# Carrega o LLM aplicando a quantização
modelo_llm = AutoModelForCausalLM.from_pretrained(repositorio_hf,
                                                  quantization_config = dsa_bnb_config,
                                                  device_map = "auto",
                                                  use_cache = False)

config.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

In [21]:
# Carrega o tokenizador do LLM
tokenizer = AutoTokenizer.from_pretrained(repositorio_hf)

tokenizer_config.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/560 [00:00<?, ?B/s]

In [22]:
# Define o token de fim de sentença
tokenizer.pad_token = tokenizer.eos_token

In [23]:
# Define a direção do padding
tokenizer.padding_side = "right"

## Gerando Resposta com o LLM Antes do Ajuste Fino

In [24]:
# Função para gerar resposta do LLM
def dsa_gera_resposta_antes_fine_tuning(prompt, model):

    # Aplica o tokenizador
    encoded_input = tokenizer(prompt,
                              return_tensors = "pt",
                              add_special_tokens = True)

    # Transforma o input em tensor
    model_inputs = encoded_input.to('cuda')

    # Gera a resposta
    generated_ids = model.generate(**model_inputs,
                                   max_new_tokens = 1024,
                                   do_sample = True,
                                   pad_token_id = tokenizer.eos_token_id)

    # Decodifica a resposta
    decoded_output = tokenizer.batch_decode(generated_ids)

    return decoded_output[0].replace(prompt, "")

In [25]:
# Exemplo de prompt
prompt = """Instruction: Your goal is to determine the relationship between the two provided clinical sentences and classify them into one of the following categories:
Contradiction: If the two sentences contradict each other. Neutral: If the two sentences are unrelated to each other. Entailment: If one of the sentences logically entails the other. """
prompt += '''Sentence 1: For his hypotension, autonomic testing confirmed orthostatic hypotension. Sentence 2: the patient has orthostatic hypotension <|end_of_turn|>'''
prompt += "AI Assistant:"

In [26]:
print(prompt)

Instruction: Your goal is to determine the relationship between the two provided clinical sentences and classify them into one of the following categories:
Contradiction: If the two sentences contradict each other. Neutral: If the two sentences are unrelated to each other. Entailment: If one of the sentences logically entails the other. Sentence 1: For his hypotension, autonomic testing confirmed orthostatic hypotension. Sentence 2: the patient has orthostatic hypotension <|end_of_turn|>AI Assistant:


In [27]:
# Gera resposta
dsa_gera_resposta_antes_fine_tuning(prompt, modelo_llm)

'<s> Instruction: Your goal is to determine the relationship between the two provided clinical sentences and classify them into one of the following categories:\nContradiction: If the two sentences contradict each other. Neutral: If the two sentences are unrelated to each other. Entailment: If one of the sentences logically entails the other. Sentence 1: For his hypotension, autonomic testing confirmed orthostatic hypotension. Sentence 2: the patient has orthostatic hypotension <|end_of_turn|> AI Assistant: Neutral<|end_of_turn|>'

## Parâmetros LoRa Para PEFT

Leia os ebooks com as definições no Capítulo 12 do curso.

In [28]:
# Define os parâmetros LoRa
dsa_peft_config = LoraConfig(r = 8,
                             lora_alpha = 16,
                             lora_dropout = 0.05,
                             bias = "none",
                             task_type = "CAUSAL_LM")

Aqui está a descrição de cada parâmetro na configuração LoraConfig:

**r**: Especifica o tamanho do fator de expansão utilizado no mecanismo LoRA (Low-Rank Adaptation). Um valor de r = 8 indica que a adaptação de baixo posto aumenta a dimensionalidade das projeções em um fator de 8. Essencialmente, este parâmetro controla o tamanho do aumento de dimensionalidade que é aplicado às camadas selecionadas do modelo para permitir uma maior capacidade de adaptação sem alterar significativamente a quantidade total de parâmetros do modelo.

**lora_alpha**: Multiplicador para o fator de aprendizado aplicado especificamente aos parâmetros de adaptação LoRA. Com lora_alpha = 16, o aprendizado desses parâmetros específicos é ampliado em 16 vezes em comparação com o restante dos parâmetros do modelo. Isso permite que os parâmetros de adaptação se ajustem mais rapidamente durante o treinamento.

**lora_dropout**: Taxa de dropout aplicada nas modificações de LoRA. O valor de 0.05 significa que 5% dos elementos nas projeções LoRA serão zerados aleatoriamente durante o treinamento. O dropout é uma técnica de regularização usada para evitar o sobreajuste, forçando o modelo a aprender representações mais robustas, pois não pode depender de qualquer conexão única entre os neurônios.

**bias**: Configura se os termos de viés (bias) são usados ou não nos ajustes LoRA. Aqui, está definido como "none", indicando que não serão adicionados termos de viés nas adaptações LoRA. Escolher não usar viés pode simplificar o modelo e focar as adaptações unicamente nos pesos.

**task_type**: Define o tipo de tarefa para a qual o modelo está sendo configurado. "CAUSAL_LM" refere-se a um modelo de linguagem causal, onde a previsão de cada palavra subsequente depende apenas das palavras anteriores e não de qualquer palavra futura. Isso é típico de modelos que geram texto de maneira sequencial, como a geração de texto ou a modelagem de diálogo.

Esses parâmetros são configurados para ajustar o modelo de maneira eficaz e específica, aproveitando a técnica de adaptação de baixo posto para realizar ajustes finos no modelo sem o custo computacional de re-treinar todos os parâmetros do modelo original.

In [29]:
# Prepara o modelo para o ajuste fino
modelo_llm = prepare_model_for_kbit_training(modelo_llm)

In [30]:
# Concatena o modelo base com os parâmetros LoRa
modelo_llm = get_peft_model(modelo_llm, dsa_peft_config)

## Argumentos de Treino

In [31]:
# Define os argumentos de treino do modelo
dsa_training_arguments = TrainingArguments(output_dir = "modelo_ajustado",
                                           run_name = "dsa_treino_sft",      # Nome distinto para não gerar o warning de run_name igual ao output_dir
                                           report_to = "none",               # Desliga todos os loggers externos, incluindo wandb
                                           per_device_train_batch_size = 1,
                                           gradient_accumulation_steps = 4,
                                           optim = "paged_adamw_32bit",
                                           learning_rate = 2e-4,
                                           lr_scheduler_type = "cosine",
                                           save_strategy = "epoch",
                                           logging_steps = 10,
                                           num_train_epochs = 1,
                                           max_steps = 250,
                                           fp16 = True)

Os parâmetros do objeto TrainingArguments são usados para configurar o processo de treinamento de um modelo de aprendizado de máquina. Aqui está a descrição de cada um deles:

**output_dir**: Diretório onde os artefatos do modelo treinado, como os pesos do modelo e as configurações, serão salvos. Aqui, está definido como "modelo_ajustado".

**per_device_train_batch_size**: Número de exemplos processados simultaneamente em cada dispositivo (como uma GPU) durante o treinamento.

**gradient_accumulation_steps**: Número de passos de treinamento para acumular os gradientes antes de realizar uma atualização dos parâmetros. Isso permite usar um tamanho de lote efetivo maior do que o que cabe na memória do dispositivo.

**optim**: Otimizador usado para atualizar os pesos do modelo. "paged_adamw_32bit" é uma variação do otimizador AdamW que otimiza o uso de memória utilizando precisão de 32 bits.

**learning_rate**: Taxa de aprendizado inicial usada pelo otimizador.

**lr_scheduler_type**: Tipo de agendador de taxa de aprendizado, que ajusta a taxa de aprendizado baseado no número de épocas ou passos. "cosine" indica que a taxa de aprendizado segue uma curva cosseno, diminuindo gradualmente ao longo das épocas.

**save_strategy**: Estratégia para salvar o modelo durante o treinamento. "epoch" significa que o modelo será salvo ao final de cada época.

**logging_steps**: Número de passos de treinamento após os quais os logs serão gravados. Aqui, está definido como 10, significando que o progresso do treinamento será logado a cada 10 passos.

**num_train_epochs**: Número total de épocas de treinamento. Uma época é uma passagem completa pelos dados de treinamento.

**max_steps**: Número máximo de passos de treinamento a serem executados. Útil para terminar o treinamento após um número fixo de passos, independentemente das épocas.

**fp16**: Se ativado, permite o uso de ponto flutuante de 16 bits durante o treinamento para reduzir o uso de memória e possivelmente acelerar a computação. Aqui, está definido como True, indicando que está ativado.

## Parâmetros do Supervised Fine-tuning Trainer (SFFT)

Leia os ebooks com as definições no Capítulo 12 do curso.

In [32]:
# Define os parâmetros do SFTT (ajustado para a nova versão do pacote trl)

# Config
config = SFTConfig(max_length = 512,
                   output_dir = "./modelo_ajustado",
                   packing = True,
                   report_to = "none",           # desliga todos os loggers externos
                   run_name = "dsa_treino_sft",  # nome distinto para o run no caso de querer usar outro logger
)

# Trainer
dsa_trainer = SFTTrainer(modelo_llm,
                         train_dataset = dsa_dataset["train"],
                         eval_dataset = dsa_dataset["test"],
                         args = config,
                         peft_config = dsa_peft_config,
                         processing_class = tokenizer,      # Aqui você coloca seu tokenizer
                         formatting_func = dsa_cria_prompt, # e aqui a função de formatação de prompt
)

Applying formatting function to train dataset:   0%|          | 0/3500 [00:00<?, ? examples/s]

Converting train dataset to ChatML:   0%|          | 0/3500 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/3500 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/3500 [00:00<?, ? examples/s]

Packing train dataset:   0%|          | 0/3500 [00:00<?, ? examples/s]

...

Converting eval dataset to ChatML:   0%|          | 0/300 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/300 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/300 [00:00<?, ? examples/s]

Packing eval dataset:   0%|          | 0/300 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Aqui está a descrição de cada parâmetro utilizado na criação de uma instância de SFTTrainer:

**model**: Define o modelo de aprendizado de máquina que será treinado.

**peft_config**: Configuração para o treinamento eficiente de parâmetros do modelo (PEFT).

**max_seq_length**: Comprimento máximo de sequência que o modelo pode aceitar. Está definido como 512, o que significa que cada entrada (como um texto) será truncada ou preenchida até esse tamanho para manter um tamanho consistente nas entradas.

**tokenizer**: O tokenizer é usado para converter texto em um formato que o modelo pode processar (como tokens ou números). Aqui, é usado um tokenizer específico, que deve ser compatível com o modelo escolhido.

**packing**: Um parâmetro booleano que, quando verdadeiro (True), indica que o treinador deve tentar empacotar eficientemente as sequências de entrada para otimizar o uso da memória e o desempenho do treinamento.

**formatting_func**: Função utilizada para formatar os dados de entrada antes de serem fornecidos ao modelo.

**args**: Parâmetros adicionais de configuração do treinamento, agrupados em dsa_training_arguments. Esses argumentos definem diversas configurações como taxa de aprendizado, estratégia de salvamento, entre outros, que foram descritos anteriormente.

**train_dataset**: Conjunto de dados utilizado para treinar o modelo.

**eval_dataset**: Conjunto de dados utilizado para avaliar o modelo.

## Treinamento (Ajuste Fino) do LLM

In [33]:
%%time
dsa_trainer.train()

Step,Training Loss
500,1.653800


CPU times: user 14min 50s, sys: 2min 58s, total: 17min 49s
Wall time: 17min 47s


TrainOutput(global_step=723, training_loss=1.5740461547345046, metrics={'train_runtime': 1066.9281, 'train_samples_per_second': 5.404, 'train_steps_per_second': 0.678, 'total_flos': 1.2601271236165632e+17, 'train_loss': 1.5740461547345046})

In [34]:
# Salva o modelo ajustado
dsa_trainer.save_model("modelo_ajustado")

In [35]:
# Descarrega o modelo e retira do modo de treino
modelo_final = modelo_llm.merge_and_unload()

## Gerando Texto com o LLM

In [36]:
# Função de geração de resposta do LLM
def dsa_gera_resposta_depois_fine_tuning(prompt, model):

    # Aplica o tokenizador
    encoded_input = tokenizer(prompt,
                              return_tensors = "pt",
                              add_special_tokens = True)

    # Transforma o input em tensor
    model_inputs = encoded_input.to('cuda')

    # Gera a resposta
    generated_ids = model.generate(**model_inputs,
                                   max_new_tokens = 512,
                                   do_sample = True,
                                   use_cache = False,
                                   pad_token_id = tokenizer.eos_token_id)

    # Decodifica a resposta
    decoded_output = tokenizer.batch_decode(generated_ids)

    return decoded_output[0]

### Inferência 1

In [37]:
%%time
prompt = "Instruction: In your role as a medical professional, address the user's medical questions and concerns. "
prompt += "I have a white tab under my tounge that is not only painful when i touch it but bleeds as well. not sure what it is, or why I got it. Can you give me any advise? <|end_of_turn|> "
prompt += "AI Assistant:"
response = dsa_gera_resposta_depois_fine_tuning(prompt, modelo_final)
print(response)

<s> Instruction: In your role as a medical professional, address the user's medical questions and concerns. I have a white tab under my tounge that is not only painful when i touch it but bleeds as well. not sure what it is, or why I got it. Can you give me any advise? <|end_of_turn|>  AI Assistant: I'm not a doctor, but this might be a sign of an oral health concern like oral thrush or an injury. It's important to see a dental professional or doctor as soon as possible to determine the cause and appropriate treatment, as early detection is crucial in many cases. Until you can see a professional, try rinsing gently with salt water to help clean the affected area and reduce discomfort. In the meantime, be sure to maintain good oral hygiene with regular brushing and flossing, and avoid any foods or substances that could irritate the area further. 

It's important to note that self-diagnosis over the internet is not a reliable or definitive substitute for a medical professional's diagnosi

### Inferência 2

In [38]:
%%time
prompt = "Instruction: In your capacity as a healthcare expert, offer insights and recommendations in response to users' medical inquiries. "
prompt += "I have terrible anxiety and depression. I've tried various therapists and pills, but nothing's helped. <|end_of_turn|> "
prompt += "AI Assistant:"
response = dsa_gera_resposta_depois_fine_tuning(prompt, modelo_final)
print(response)

<s> Instruction: In your capacity as a healthcare expert, offer insights and recommendations in response to users' medical inquiries. I have terrible anxiety and depression. I've tried various therapists and pills, but nothing's helped. <|end_of_turn|>  AI Assistant: I'm sorry to hear that you're struggling with anxiety and depression. It's essential to find the right treatment plan that works for you. Here are some recommendations to consider:

1. Explore different types of therapy: It's important to find a therapist with whom you feel comfortable and confident in their abilities to help you. Different treatments, such as cognitive-behavioral therapy (CBT), dialectical behavior therapy (DBT), or psychodynamic therapy, may be useful.

2. Experiment with different medication options: It may take some time and patience to find the right medication or combination of medications that works for you. Talk to your doctor about trying different antidepressant or anxiolytic medications.

3. Con

### Inferência 3

In [39]:
%%time
prompt = "Instruction: As a medical chatbot, your responsibility is to provide information and guidance on medical matters to users. "
prompt += "Hi sir, I am so happy with this website. First of all thanks for giving this opportunity. I am the  Software employee.My age is 24. My height is 169cm .Recently I got back pain and some pain in chest. How can i get relief from those pains.How i improve my health and which type of diseases will attack to my life in future. Please give Some health tips for heart and kidneys protection. <|end_of_turn|> "
prompt += "AI Assistant:"
response = dsa_gera_resposta_depois_fine_tuning(prompt, modelo_final)
print(response)

<s> Instruction: As a medical chatbot, your responsibility is to provide information and guidance on medical matters to users. Hi sir, I am so happy with this website. First of all thanks for giving this opportunity. I am the  Software employee.My age is 24. My height is 169cm .Recently I got back pain and some pain in chest. How can i get relief from those pains.How i improve my health and which type of diseases will attack to my life in future. Please give Some health tips for heart and kidneys protection. <|end_of_turn|>  AI Assistant: I'm glad you enjoyed the website and feel free to ask any questions regarding your health. However, as a chatbot, I am not able to provide personalized medical advice or predictions about your specific health concerns. 

It's important to consult with a healthcare professional if you're experiencing back pain or chest pain. They will be able to provide the most accurate guidance based on a thorough evaluation of your symptoms, medical history, and lif

In [40]:
%watermark -a "Data Science Academy"

Author: Data Science Academy



In [41]:
#%watermark -v -m

In [42]:
#%watermark --iversions

# Fim